In [23]:
import numpy as np
import pandas as pd
import plotly.express as px

In [75]:
amrhet = pd.read_csv('topics_amrhet_oba.csv')
amrhet.date = pd.to_datetime(amrhet.date, format='%Y-%m-%d')
speeches101 = pd.read_csv('topics_speeches_oba.csv')
speeches101.date = pd.to_datetime(speeches101.date, format='%Y-%m-%d')
topics = pd.read_csv('./topics/topics_9.csv')
topics

,topic,topic_name
0,"0.017*""energy"" + 0.014*""business"" + 0.013*""com...",energy
1,"0.015*""peace"" + 0.013*""freedom"" + 0.013*""democ...",democracy
2,"0.018*""student"" + 0.016*""school"" + 0.013*""educ...",education
3,"0.017*""veteran"" + 0.017*""intelligence"" + 0.016...",veterans
4,"0.019*""tax"" + 0.016*""pay"" + 0.015*""cut"" + 0.01...",taxes
5,"0.036*""iraq"" + 0.022*""terrorist"" + 0.019*""mili...",economy
6,"0.047*""nuclear"" + 0.043*""iran"" + 0.025*""israel...",security
7,"0.014*""love"" + 0.012*""god"" + 0.007*""faith"" + 0...",civil rights
8,"0.030*""health"" + 0.029*""care"" + 0.022*""insuran...",health care


In [77]:
# let's start with 
cutoff = pd.to_datetime('2008-11-06', format='%Y-%m-%d')
amrhet = amrhet.query('date > @cutoff')
speeches101 = speeches101.query('date > @cutoff')
amrhet = amrhet.drop(['file', 'topics'], axis=1)
amrhet_tall = pd.melt(amrhet, id_vars='date',value_vars=['0','1','2','3','4','5','6','7','8'],
                var_name='topic')
speeches101 = speeches101.drop(['file', 'topics'], axis=1)
speeches101_tall = pd.melt(speeches101, id_vars='date',value_vars=['0','1','2','3','4','5','6','7','8'],
                var_name='topic')

<A HREF="https://plotly.com/python/linear-fits/">Plotly trendlines and linear fits</A>

In [59]:
fig = px.scatter(amrhet, x="date", y="0", trendline="lowess", trendline_color_override="red")
fig.show()

In [97]:
# OLS trendline
fig = px.scatter(amrhet_tall, x="date", y="value", facet_row="topic", 
                 trendline="ols", trendline_color_override="red", height=1600)
fig.show()

In [64]:
# Moving Average
window_size = 30
fig = px.scatter(amrhet_tall, x="date", y="value", facet_row="topic", 
                 trendline="rolling", trendline_options=dict(window=window_size),
                 trendline_color_override="red", height=1600)
fig.show()

In [58]:
# On the 100 speeches used for comparing to newspapers and Bush
fig = px.scatter(speeches101_tall, x="date", y="value", facet_row="topic", 
                 trendline="ols", trendline_color_override="red", height=1600)
fig.show()

In [93]:
ar = amrhet.drop('text', axis=1)
ar = ar.set_index('date')
ar.columns = list(topics.topic_name)
ar = ar.rename_axis("topics", axis="columns")
ar.head(3)

topics,energy,democracy,education,veterans,taxes,economy,security,civil rights,health care
date,,,,,,,,,
2008-11-07,0.135662,0.191097,0.092927,0.0,0.000000,0.062935,0.046698,0.393113,0.076525
2008-11-18,0.185452,0.000000,0.214408,0.0,0.000000,0.116554,0.134147,0.329958,0.017115
2008-11-24,0.100931,0.162856,0.026403,0.0,0.081991,0.087480,0.000000,0.539544,0.000000


In [96]:
fig = px.scatter(ar, trendline="rolling", trendline_options=dict(window=30),
                title="Topic Moving Average")
fig.data = [t for t in fig.data if t.mode == "lines"]
fig.update_traces(showlegend=True) #trendlines have showlegend=False by default
fig.show()